# Evaluation

This notebook shows how to obtain a more fine-grained assessment of the tree detection and segmentation performance.

In [ ]:
import torch
import os
import matplotlib.pyplot as plt
from tree_learn.util import load_data, juxtapose, plot_evaluation_results_segments
%load_ext autoreload
%autoreload 2

In [2]:
os.chdir(os.path.dirname(os.path.abspath("__file__")))
gt_forest_path = '../../data/benchmark/L1W_voxelized01_for_eval.laz' # path to ground truth forest used for evaluation
pred_forest_path = '../../data/pipeline/L1W/results/full_forest/evaluation/pred_forest_propagated_to_gt_pointcloud.laz' # path to predicted forest (with points propagated to GT point cloud to enable visualizations)
evaluation_results_path = '../../data/pipeline/L1W/results/full_forest/evaluation/evaluation_results.pt' # path to instance evaluation object (change according to where your evaluation was performed)

# loading data

In [3]:
evaluation_results = torch.load(evaluation_results_path)
gt_forest = load_data(gt_forest_path)
pred_forest = load_data(pred_forest_path)

coords = gt_forest[:, :3]
instance_labels = gt_forest[:, 3]
instance_preds = pred_forest[:, 3]

# Instance detection
Apart from looking at the aggregated metrics displayed in the log, it is also possible to display more fine-grained information about the non-matched predictions or ground truths:

In [ ]:
# non-matched predictions
print(f'non-matched predictions: {evaluation_results["detection_results"]["non_matched_preds_filtered"]}') 
print(f'corresponding ground truth of non-matched predictions: {evaluation_results["detection_results"]["non_matched_preds_corresponding_gt_filtered"]}')

# non-matched ground truths
print(f'non-matched ground truths: {evaluation_results["detection_results"]["non_matched_gts"]}')
print(f'corresponding prediction of non-matched ground truths: {evaluation_results["detection_results"]["non_matched_gts_corresponding_pred"]}')
print(f'matched ground truth tree for that prediction: {evaluation_results["detection_results"]["non_matched_gts_corresponding_other_tree"]}')

Using these label correspondences, it is possible to visualize concrete cases of omission and comission errors. For example, in the code below, change the pred_num and gt_num to a pair taken from "non-matched prediction" and "corresponding ground truth of non-matched predictions" to visualize commission errors. You can toggle the visualization of the juxtaposed point clouds by clicking on the legend.

In [ ]:
pred_num = 62
gt_num = 190
coords_pred = coords[instance_preds == pred_num]
coords_gt = coords[instance_labels == gt_num]
juxtapose(coords_pred, coords_gt, 'pred', 'gt')

# Instance segmentation
Apart from the aggregated metrics displayed in the log, it is possible to look at the segmentation of individual trees:

In [ ]:
evaluation_results['segmentation_results']['no_partition']

Furthermore, you can visualize the results of the xy and z partition evaluation. You need to adjust the parameters of the plot according to the results of your method:

In [ ]:
color = '#010003'
fontsize=7
fig_size=(4,1)
y_range=[60, 100]
x_label = "Center \u2192 Outer branches"
fig, axs = plt.subplots(1, 3, figsize=fig_size)
plt.tight_layout()


# precision
values = evaluation_results['segmentation_results']['xy_partition'].iloc[:, 2:12].mean(0).to_numpy() * 100
axs[0] = plot_evaluation_results_segments(axs[0], values, fontsize=fontsize, measure="Precision in %", y_range=y_range, color=color, x_label=x_label)

# recall
values = evaluation_results['segmentation_results']['xy_partition'].iloc[:, 12:22].mean(0).to_numpy() * 100
axs[1] = plot_evaluation_results_segments(axs[1], values, fontsize=fontsize, measure="Recall in %", y_range=y_range, color=color, x_label=x_label)

# coverage
values = evaluation_results['segmentation_results']['xy_partition'].iloc[:, 22:32].mean(0).to_numpy() * 100
axs[2] = plot_evaluation_results_segments(axs[2], values, fontsize=fontsize, measure="Coverage in %", y_range=y_range, color=color, x_label=x_label)

fig.subplots_adjust(wspace=0.45, left=0, right=1, bottom=0.15, top=1)
plt.show()

In [ ]:
color = '#010003'
fontsize=7
fig_size=(4,1)
y_range=[60, 100]
x_label = "Bottom \u2192 Top"
fig, axs = plt.subplots(1, 3, figsize=fig_size)
plt.tight_layout()


# precision
values = evaluation_results['segmentation_results']['z_partition'].iloc[:, 2:12].mean(0).to_numpy() * 100
axs[0] = plot_evaluation_results_segments(axs[0], values, fontsize=fontsize, measure="Precision in %", y_range=y_range, color=color, x_label=x_label)

# recall
values = evaluation_results['segmentation_results']['z_partition'].iloc[:, 12:22].mean(0).to_numpy() * 100
axs[1] = plot_evaluation_results_segments(axs[1], values, fontsize=fontsize, measure="Recall in %", y_range=y_range, color=color, x_label=x_label)

# coverage
values = evaluation_results['segmentation_results']['z_partition'].iloc[:, 22:32].mean(0).to_numpy() * 100
axs[2] = plot_evaluation_results_segments(axs[2], values, fontsize=fontsize, measure="Coverage in %", y_range=y_range, color=color, x_label=x_label)

fig.subplots_adjust(wspace=0.45, left=0, right=1, bottom=0.15, top=1)
plt.show()